# Exercise Book 4

**Covering the materials of Chapter 11.**
Topics: vector spatial data management with geopandas

In the attached `data` folder the following attached datasets are given for this assignment:
 - `hungary_admin_8.shp`, containing the city level administrative boundaries of Hungary. (Data source: [*OpenStreetMap*](https://data2.openstreetmap.hu/hatarok/))
 - `hungary_population_2020.csv`, containing the population of Hungarian cities on 2020 January 1. (Data source: [*Hungarian Government*](https://www.nyilvantarto.hu/hu/statisztikak))
 - `hungary_population_2011.csv`, containing the population of Hungarian cities on 2011 January 1. (Data source: [*Hungarian Government*](https://www.nyilvantarto.hu/hu/statisztikak))

*Note:* in the CSV files the columns are delimited with `;` characters (instead of the default `,`).

---

## Task 1

Write a program that creates a thematic map for Hungary based on the adminstrative boundaries of the cities and their population in 2020.  
(Use the *All population* field from the CSV file.)

In [5]:
import pandas as pd
import geopandas as gpd

# Read the datasets
cities = gpd.read_file('../data/hungary_admin_8.shp')
cities = cities[['NAME', 'geometry']]
cities.set_index('NAME', inplace=True)

population_2020 = pd.read_csv('../data/hungary_population_2020.csv', delimiter = ';')
population_2020.set_index('City', inplace=True)

In [3]:
# Add the population DataSeries to the cities "manually"
df = cities.copy()
df['All population'] = [None] * len(cities)

# Get the indexes which are present in both DataFrames
indexes = set(cities.index) & set(population_2020.index)
for index in indexes:
    df.loc[index, 'All population'] = population_2020.loc[index]['All population']

display(df)

,geometry,All population
NAME,,
Murakeresztúr,"POLYGON ((1875811.200 5837364.810, 1875829.320...",1712
Tótszerdahely,"POLYGON ((1865447.010 5842664.860, 1865626.780...",1081
Molnári,"POLYGON ((1871422.780 5840886.420, 1871468.690...",689
Semjénháza,"POLYGON ((1874690.000 5845206.400, 1874749.090...",566
Felsőszölnök,"POLYGON ((1793789.650 5920727.330, 1793969.030...",578
...,...,...
Milota,"POLYGON ((2530430.020 6120050.180, 2530441.900...",998
Tiszabecs,"POLYGON ((2535824.870 6121698.150, 2535957.370...",1550
Garbolc,"POLYGON ((2543379.140 6098625.170, 2543444.730...",150


In [ ]:
# This can be done in an easier and more efficient way with pandas' merge() function
df = cities.merge(population_2020, left_index=True, right_index=True)
display(df)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Create the plot
df.plot(column='All population', figsize=[20,10], legend=True, cmap='YlOrRd', scheme='quantiles', k=10)
plt.show()

## Task 2

Write a program that adds the population data for 2011 and 2020 to the Shapefile as new scalar fields to each city; and save it as a new Shapefile.

In [ ]:
population_2011 = pd.read_csv('../data/hungary_population_2011.csv', delimiter = ';')
population_2011.set_index('City', inplace=True)

df = df.merge(population_2011, left_index=True, right_index=True, suffixes=[' 2020', ' 2011'])
df.rename(columns={'County 2020':'County'}, inplace=True)
del df['County 2011']
display(df)

In [ ]:
# Save it to file
df.to_file('hungary_population.shp')

## Task 3

Write a program that creates a thematic map for Hungary based on the adminstrative boundaries of the cities and their population change between 2011 and 2020.

In [ ]:
df['Population difference'] = df['All population 2020'] - df['All population 2011']

ax = df.plot(column='Population difference', figsize=[20,10], legend=True, cmap='bwr', vmin=-5000, vmax=5000)
ax.set_facecolor("lightgray") # background color
plt.show()

**Optional:** add a raster basemap with *contextily*.

In [ ]:
# How to install: conda install -c conda-forge contextily
# How to use: https://contextily.readthedocs.io/en/latest/
import contextily as ctx

# Verify CRS, must be Web Mercator (EPSG:3857) to add a base map with the contextily module.
print(df.crs)
if df.crs == 'epsg:3857':
    ax = df.plot(column='Population difference', figsize=[20,10], legend=True, cmap='bwr', vmin=-5000, vmax=5000, alpha=0.85)
    ctx.add_basemap(ax)
    ax.set_axis_off()
    plt.show()
else:
    print('CRS must be EPSG:3857, instead {0} was given'.format(df.crs))

## Task 4

Write a program that creates a thematic map for Hungary based on the adminstrative boundaries of the cities and their population density in 2020.

In [ ]:
df_eov = df.to_crs('EPSG:23700') # EOV is EPSG:23700 
df['Area'] = df_eov.area / 10**6
df['Density 2020'] = df['All population 2020'] / df['Area'] 
display(df)

In [ ]:
df.plot(column='Density 2020', figsize=[20,10], legend=True, cmap='YlOrRd', scheme='quantiles', k=10)
plt.show()